<a href="https://colab.research.google.com/github/rudraser/Fluter-BasicApps/blob/main/ResNet74_actual.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torchvision # torch package for vision related things
import torch.nn.functional as F  # Parameterless functions, like (some) activation functions
import torchvision.datasets as datasets  # Standard datasets
import torchvision.transforms as transforms  # Transformations we can perform on our dataset for augmentation
from torch import optim  # For optimizers like SGD, Adam, etc.
from torch import nn  # All neural network modules
from torch.utils.data import DataLoader  # Gives easier dataset managment by creating mini batches etc.
from tqdm import tqdm  # For nice progress bar!
device = (torch.device('cuda') if torch.cuda.is_available()
          else torch.device('cpu'))
print(f"Training on device {device}.")

Training on device cuda.


In [2]:

from matplotlib import pyplot as plt
import numpy as np
import collections


import torch.nn as nn

import torch.optim as optim

In [3]:
from torchvision import datasets, transforms
data_path = '/data-unversioned/p1ch6/'
cifar10 = datasets.CIFAR10(
    data_path, train=True, download=True,
    transform=transforms.Compose([
        transforms.RandomCrop(size=[32,32], padding=4),                        
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
    ]))


Extracting /data-unversioned/p1ch6/cifar-10-python.tar.gz to /data-unversioned/p1ch6/


In [4]:
cifar10_val = datasets.CIFAR10(
    data_path, train=False, download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
    ]))

Files already downloaded and verified


In [5]:
class resblock(nn.Module):
    def __init__(self, in_channels, mid_channels,identity_downsample=None, stride=1):
        super(resblock, self).__init__()
        self.expansion = 4
        self.conv1 = nn.Conv2d(in_channels, mid_channels, kernel_size=1, stride=1, padding=0, bias=False)
        self.bn1 = nn.BatchNorm2d(mid_channels)
        self.conv2 = nn.Conv2d(mid_channels,mid_channels,kernel_size=3,stride=stride,padding=1,bias=False)
        self.bn2 = nn.BatchNorm2d(mid_channels)
        self.conv3 = nn.Conv2d(mid_channels,mid_channels * self.expansion,kernel_size=1,stride=1,padding=0,bias=False)
        self.bn3 = nn.BatchNorm2d(mid_channels * self.expansion)
        self.relu = nn.ReLU()
        self.identity_downsample = identity_downsample
        self.stride = stride

    def forward(self, x):
        identity = x.clone()

        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)
        x = self.conv3(x)
        x = self.bn3(x)

        if self.identity_downsample is not None:
            identity = self.identity_downsample(identity)

        x += identity
        x = self.relu(x)
        return x

In [6]:
class ResNet(nn.Module):
    def __init__(self, resblock, layers, image_channels = 3, num_classes = 10):
        super(ResNet, self).__init__()
        self.in_channels = 64
        self.conv1 = nn.Conv2d(image_channels, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        
        self.layer1 = self._create_layer(resblock, layers[0], mid_channels=64, stride=1)
        self.layer2 = self._create_layer(resblock, layers[1], mid_channels=128, stride=2)
        self.layer3 = self._create_layer(resblock, layers[2], mid_channels=256, stride=2)
        self.layer4 = self._create_layer(resblock, layers[3], mid_channels=512, stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc1 = nn.Linear(512 * 4, 32)
        self.fc2 = nn.Linear(32, 16)
        self.fc3 = nn.Linear(16, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)

        return x

    def _create_layer(self, resblock, num_residual_blocks, mid_channels, stride):
        identity_downsample = None
        new_layers = []

        # Either if we half the input space for ex, 56x56 -> 28x28 (stride=2), or channels changes
        # we need to adapt the Identity (skip connection) so it will be able to be added
        # to the layer that's ahead
        if stride != 1 or self.in_channels != mid_channels * 4:
            identity_downsample = nn.Sequential(
                nn.Conv2d(
                    self.in_channels,
                    mid_channels * 4,
                    kernel_size=1,
                    stride=stride,
                    bias=False
                ),
                nn.BatchNorm2d(mid_channels * 4),
            )

        new_layers.append(
            resblock(self.in_channels, mid_channels, identity_downsample, stride)
        )

        # The expansion size is always 4 for ResNet 50,101,152
        self.in_channels = mid_channels * 4

        # For example for first resnet layer: 256 will be mapped to 64 as intermediate layer,
        # then finally back to 256. Hence no identity downsample is needed, since stride = 1,
        # and also same amount of channels.
        for i in range(num_residual_blocks - 1):
            new_layers.append(resblock(self.in_channels, mid_channels))

        return nn.Sequential(*new_layers)



In [7]:
import datetime

def training_loop(n_epochs, optimizer, model, loss_fn, train_loader):
    for epoch in range(1, n_epochs + 1):
        loss_train = 0.0
        for imgs, labels in train_loader:
            imgs = imgs.to(device=device)  # <1>
            labels = labels.to(device=device)
            outputs = model(imgs)
            loss = loss_fn(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            loss_train += loss.item()

        
        print('{} Epoch {}, Training loss {}'.format(
            datetime.datetime.now(), epoch,
            loss_train / len(train_loader)))

In [8]:
train_loader = torch.utils.data.DataLoader(cifar10, batch_size=64,
                                           shuffle=False)

model =  ResNet(resblock,[3, 6, 12, 3]).to(device=device)# <1>
optimizer = optim.SGD(model.parameters(), lr=1e-2)
loss_fn = nn.CrossEntropyLoss()

In [9]:
numel_list = [p.numel() for p in model.parameters()]
sum(numel_list), numel_list

(30837530,
 [9408,
  64,
  64,
  4096,
  64,
  64,
  36864,
  64,
  64,
  16384,
  256,
  256,
  16384,
  256,
  256,
  16384,
  64,
  64,
  36864,
  64,
  64,
  16384,
  256,
  256,
  16384,
  64,
  64,
  36864,
  64,
  64,
  16384,
  256,
  256,
  32768,
  128,
  128,
  147456,
  128,
  128,
  65536,
  512,
  512,
  131072,
  512,
  512,
  65536,
  128,
  128,
  147456,
  128,
  128,
  65536,
  512,
  512,
  65536,
  128,
  128,
  147456,
  128,
  128,
  65536,
  512,
  512,
  65536,
  128,
  128,
  147456,
  128,
  128,
  65536,
  512,
  512,
  65536,
  128,
  128,
  147456,
  128,
  128,
  65536,
  512,
  512,
  65536,
  128,
  128,
  147456,
  128,
  128,
  65536,
  512,
  512,
  131072,
  256,
  256,
  589824,
  256,
  256,
  262144,
  1024,
  1024,
  524288,
  1024,
  1024,
  262144,
  256,
  256,
  589824,
  256,
  256,
  262144,
  1024,
  1024,
  262144,
  256,
  256,
  589824,
  256,
  256,
  262144,
  1024,
  1024,
  262144,
  256,
  256,
  589824,
  256,
  256,
  262144,
  

In [ ]:
training_loop(  
    n_epochs = 35,
    optimizer = optimizer,
    model = model,
    loss_fn = loss_fn,
    train_loader = train_loader,
)

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


2021-07-30 13:20:00.937812 Epoch 1, Training loss 1.8669263214406455
2021-07-30 13:21:04.451455 Epoch 2, Training loss 1.5535072237634293
2021-07-30 13:22:07.980964 Epoch 3, Training loss 1.408302397679185
2021-07-30 13:23:11.579371 Epoch 4, Training loss 1.2936536613327767


In [1]:
train_loader = torch.utils.data.DataLoader(cifar10, batch_size=64,
                                           shuffle=False)
val_loader = torch.utils.data.DataLoader(cifar10_val, batch_size=64,
                                         shuffle=False)

def validate(model, train_loader, val_loader):
    for name, loader in [("train", train_loader), ("val", val_loader)]:
        correct = 0
        total = 0

        with torch.no_grad():  # <1>
            for imgs, labels in loader:
                imgs = imgs.to(device=device)
                labels = labels.to(device=device)
                outputs = model(imgs)
                _, predicted = torch.max(outputs, dim=1) # <2>
                total += labels.shape[0]  # <3>
                correct += int((predicted == labels).sum())  # <4>

        print("Accuracy {}: {:.2f}".format(name , correct / total))

validate(model, train_loader, val_loader)

NameError: ignored